# Energy-based Models
## Andreas Kirsch, Oct 2020

## Energy

We assign an energy to $E(x)$ every state $x$ of a model:

$$E(x): \mathcal{X} \rightarrow \mathbb{R}$$

The intuition is that like in physics, our model describes a dynamic system in which states of low energy are preferred. Specifically, local minima form stable attractors. 

We describe our tasks in a way that low energies encode preferred states. 

The energy function has no "zero" point. It is a relative quantity, just like in physics: we only care about the difference in energy between different states.

<img src="images/energy_function.png" width="50%"/>

## Usage
#### Classification/Prediction
The state can be (input, output):  $E(x,y): \mathcal{X} \times \mathcal{Y} \rightarrow \mathbb{R}.$
Then we are interested in:
$\arg\min_y E(x,y)$ for a given $x$.
#### Error correction
$x^* =  \arg\min E(x)$ s.t. $x \in B_\epsilon(x_0)$
#### Completion
For $x = (x_1,x_2)$, we want $x^* = \arg\min_x E(x_1, x_2)$ where $x_1$ is fixed and $x_2$ is unknown and sampled randomly.
#### Content-addressable memory (CAM)
All of these tasks (above) can be viewed as implementing content-addressable memory, which is about retrieving memory based on its (partial) content.


#### Ranking
$E(x_1) > E(x_2)$?
#### Out-of-distribution detection
Decision problem: $E(x) > \tau \Rightarrow \text{ood}(x)$

## How to turn $E(x)$ into a Probabilistic Model
We can use the Gibbs distribution for that:

$$p(x) = \frac{\exp(-E(x))}{\int \exp(-E(x)) \,dx}$$

Sometimes: $p(x) = \frac{\exp(-\beta E(x))}{\int \exp(-\beta E(x)) \,dx}$ where $\beta$ is the "inverse temperature" and only interesting for calibration purposes

We also define the partition function(al):
$$ Z_E := \int \exp(-E(x)) \,dx $$
then:
$$ p(x) = \frac{\exp(-E(x))}{Z_E} $$

$\exp(-E(x))$ ensures positivity of the density. Moreover, it is the natural option given that we only care about relative energies. There ought to be no difference if we shift energies by a constant amount:

$$p(x) = \frac{\exp(-E(x) - C)}{\int \exp(-E(x) - C) \,dx} = \frac{\exp(-C) \exp(-E(x))}{\exp(-C) \int \exp(-E(x)) \,dx} = \frac{\exp(-E(x))}{\int \exp(-E(x)) \,dx} $$

This property and the Gibbs measure might look familiar: the `softmin` function is a categorical version of it (`softmin(x) = softmax(-x))`).

## How do we train EBMs conceptually?

The optimization problem is determined by the family of energy functions $\mathcal{E}$ that is available and by a loss functional $\mathcal{L}$ and the training data $D_{train} = \{x_i\}_i$.
$$E^* = \arg\min_{E \in \mathcal{E}} \mathcal{L}(E, D_{train})$$


The loss function $\mathcal L$ can take many different forms: cross-entropy training is just one possible loss function.

We want to push down energies for samples in the training set and pull up energies for other states with low energy.

![Conceptual EBM training](images/EBM_training.png)
(from "A Tutorial on Energy-Based Learning" by LeCun, 2006)

## Cross-entropy loss, one more time 
With $ p(x) = \frac{\exp(-E(x))}{Z_E} $:

$$ \mathcal{L}(E, D_{train}) = \mathbb{E}_{\hat{p}(x)} \left [ -\log p(x) \right ] = \mathbb{E}_{\hat{p}(x)} \left [ E(x) + \log Z_E \right ] $$ with the empirical distribution $$\hat{p}(x) \propto \sum_{x' \in D_{train}} \delta_{x'}(x)$$

### Derivative

$$ d \mathcal{L}(E, D_{train}) = \mathbb{E}_{\hat{p}(x)} \left [ dE(x)  + \frac{1}{Z_E} \int \exp(-E(x')) (-dE(x')) \, dx' \right ] = \underbrace{\mathbb{E}_{\hat{p}(x)} dE(x)}_\text{Positive samples} - \underbrace{\mathbb{E}_{p(x')} dE(x')}_\text{Negative samples} $$

(If $E$ is parameterized by $\theta$, we can substitute $dE = \frac{\partial E}{\partial \theta} d\theta$.)

We push down the energy for positive samples and pull up the energy for negative samples.

## How do we train it?

$$ d \mathcal{L}(E, D_{train}) = \mathbb{E}_{\hat{p}(x)} \left [ dE(x)  + \frac{1}{Z_E} \int \exp(-E(x')) (-dE(x')) \, dx' \right ] = \underbrace{\mathbb{E}_{\hat{p}(x)} dE(x)}_\text{Positive samples} - \underbrace{\mathbb{E}_{p(x')} dE(x')}_\text{Negative samples} $$

The "positive samples" term can be easily computed using the training set. 

**The "negative samples" term is the tricky part.**

We can use different methods to sample from $p(x)$: MCMC, SGLD, HMC, etc.

### Contrastive Divergence

Start MCMC from existing training samples and only run for a few steps

Q: How does this compare to adversarial training? Add noise and follow the gradient for a few steps. 
Has anyone tried that actually?

## "Your Classifier is secretely an EBM..." by Grathwohl et al, 2019
### Or how to turn $E(x, y)$ into a Probabilistic Model

<img src="images/jem.png" style="float: right;"/>

Let's say $y$ is categorical.

$$ p(x,y) = \frac{\exp(-E(x, y))}{\sum_y \int \exp(-E(x, y)) \,dx} = \frac{\exp(-E(x, y))}{Z_E}$$

### Marginal
$$p(x) =  \frac{\sum_y \exp(-E(x, y))}{\sum_y \int \exp(-E(x, y)) \,dx} $$
$$ E(x) = - \log \sum_y \exp(-E(x, y)) $$

### Conditional
$$ p(y | x) = \frac{p(x,y)}{p(x)} = \frac{\exp(-E(x, y))}{\sum_y \exp(-E(x, y))} $$
=> $p(y|x)$ is independent of $Z_E$! We can rewrite this into a well-known expression in DNNs:
$$ p(y|x) = \text{Softmax}( -E(x, \cdot) ) $$

Overall, we train $$ \log p(x,y) = \log p(x) + \log p(y|x) $$ by training the marginal and conditional separately: the conditional $ \log p(y|x) $ can be optimized using normal softmax-cross-entropy, whereas the marginal $\log p(x)$ can be trained using SGLD. 

### After-thought

When we train $p(x)$ for a given $x$, we don't want to change $p(y|x)$, do we? We only want to shift $p(x)$ by some $C = \Delta p(x)$.

$$ E(x) + C = - \log( \exp(-C)) - \log \sum_y \exp(-E(x, y)) = -log \left ( \exp(-C) \sum_y \exp(-E(x, y)) \right ) = -\log \sum_y \exp(-(E(x,y) + C))$$

However, does this happen in practice?

## Hopfield networks

## Ising Model

We want to model the magnetic spin of atoms in a lattice. The magnetic spins interact with each other to align themselves:

* in the same direction, for ferromagnetic interactions, and
* in opposing directions, for antiferromagnetic interactions.

![Ferromagnetic material](images/Ising_ferromagnetic.png)

The Hamiltonian for the Ising Model is
$$H(\sigma)=-\sum_{\langle i j\rangle} J_{i j} \sigma_{i} \sigma_{j}$$
for discrete spin variables $\sigma_i \in \{-1, +1\}$ with interactions $J_{i,j}$.

$J_{i,j} > 0$ for ferromagnetic interaction,
$J_{i,j} < 0$ for antiferromagnetic interaction, and
$J_{i,j} = 0$ for no interaction.

## (Discrete) Hopfield networks 

*Hopfield networks are models for associative memory, so content-addressable memory.*

We model a neuron's state as either ON or OFF, $S_i \in \{-1, +1\}$.

The follow the following update rule:

$$S_{i} \leftarrow\left\{\begin{array}{ll}+1 & \text { if } \sum_{j} w_{i j} S_{j} \geq \theta_{i} \\ -1 & \text { otherwise }\end{array}\right.$$

where $w_{i j}$ specifies the weights of the synaptic connection from neuron $j$ to $i$, and $\theta_i$ is the activation threshold for neuron $i$.

This leads to the following energy function:

$$E=-\frac{1}{2} \sum_{i, j} w_{i j} S_{i} S_{j}+\sum_{i} \theta_{i} s_{i}$$

We set $\theta_i = 0$ here.

### Hebbian learning

To learn patterns $\mu = {p^\mu_i} \ in M$, we set (for $i \not = j$):
$$w_{i j} = \frac{1}{| M |} \sum_{\mu \in M} p^\mu_i p^\mu_j$$
and $w_{i i} = 0$. 

![Learnt energy function](images/hopfield_attractor.png)

## In practice

We can either start from a randomly pattern and let it converge to a stored pattern (as a generative model), or we can partially corrupt data and have the model inpaint the rest.

<div>
<img src="images/hopfield_example.png" width="45%" style="display:inline-block;" /><img src="images/hopfield_weights.png" width="45%" style="display:inline-block;"/>
</div>

(from https://github.com/takyamamoto/Hopfield-Network)

## Sources

#### Energy-based models

* LeCun, Y. et al. “A Tutorial on Energy-Based Learning.” (2006)
* Grathwohl, Will, et al. "Your classifier is secretly an energy based model and you should treat it like one." International Conference on Learning Representations. 2019.



#### Hopfield networks

* Ch. 17, "Neuronal Dynamics: From single neurons to networks and models of cognition", https://neuronaldynamics.epfl.ch/online/Ch17.S2.html
* https://neuronaldynamics-exercises.readthedocs.io/en/latest/exercises/hopfield-network.html
* https://github.com/takyamamoto/Hopfield-Network